In [37]:
import string
import sys
import os
import pandas as pd
import numpy as np
import re
from tqdm import *
tqdm.pandas()

In [38]:
Names_new= pd.read_csv("/Users/kamakshibansal/Moorfields/Distinct_name_new_count.csv")
Names=pd.DataFrame()
Names["Forename"]=Names_new["Forename"]
Names=Names.dropna(axis=0)
wordFrequency_name = dict.fromkeys(Names["Forename"].str.lower().unique().tolist(),1 )

In [39]:
Names

,Forename
0,adlane
1,admira
2,adotey
3,adrews
4,adzewnko
...,...
184529,david
184530,lane
184531,essex
184532,john


In [40]:
def clean_sentences(sentence):
    result1= re.sub(r"""["?,>}{)"\/(.\\:$!]|'(?!(?<! ')[ts])|'s""", " ", sentence.lower())
    result1= re.sub(' +', ' ', result1)
    result1= re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '',result1)
    result2= remove_hospitalname(result1)
    return result2

def clean_mr(query):
    mr_list = re.compile(r'(^ms|^ms.|^miss|^mr|^mr.|^mrs.|^miss.|^mrs|^dr.|^dr|^dear.|^dob|^number|\s*ms|\s*ms.|\s*miss|\s*mr|\s*mr.|\s*mrs.|\s*miss.|\s*mrs|\s*dr.|\s*dr|\s*dob|\s*number)\s+([^\s]+)\s*').findall(query)
    mr_list = sorted(mr_list, key = lambda k : len(k[1]), reverse=True)
    for mr_word_tuple in mr_list:
        word_comb = ' '.join(list(mr_word_tuple))
        first_word = mr_word_tuple[0]
        query = query.replace(word_comb, first_word+' ')
        
    query=re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '',query)
    return query

def clean_mr_and_remove_stopwords(query, merge_dict):
    result2 = clean_mr(query).split()
    resultwords  = [word for word in result2 if merge_dict[word]==0]
    final1= ' '.join(resultwords) 
    final2= remove_match(final1)
    final= remove_cc(final2)
#     print (final)
    return final

def remove_match(qu):
    qu= re.sub('re.*b0','XXXXXXXX', qu, flags= re.DOTALL)
    return qu

def remove_hospitalname(abc):
    result= re.sub(r"\w+(?=\s+hospital)", "", abc)
    return result

def remove_cc(qu):
    result= re.sub('original recipient.*par','XXXXXXXX', qu, flags= re.DOTALL)
    return result

In [21]:
# for i in range(1999, 2020):
#     print(i)

In [1]:
# number_of_files = 2

for i in range(1999, 2020):
    print ("Working on year {} file".format(i))
    Letters = pd.read_csv("/Users/kamakshibansal/Moorfields/Letters/Cleaned_Letters/cleaned{}.csv".format(i))
    Letters=Letters.dropna()


    b=Letters["LetterBody"].map(clean_sentences).str.lower()
    b=b.values.tolist()
    k=' '.join(b)
    z=k.split()

    z1=list(dict.fromkeys(z))
    wordFrequency = dict.fromkeys(z1,0 )
    merge_dict= { **wordFrequency,**wordFrequency_name}

    %timeit -n 1 -r 1 Letters['clean_sentence'] = Letters.LetterBody.progress_map(clean_sentences)
    %timeit -n 1 -r 1 Letters['final_sentence'] = Letters.clean_sentence.progress_map(lambda c: clean_mr_and_remove_stopwords(c, merge_dict))

    Letters= Letters.drop(["PatKey","clean_sentence", "SeqId","LetterBodyX"], axis=1)

    Letters.to_csv("/Users/kamakshibansal/Moorfields/Letters/Anonymised/Letters_new_{}.csv".format(i))
    


Working on year 1999 file


NameError: name 'pd' is not defined

In [1]:
ab= "abrasion, corneal abrasion, corneal erosion, ct, left, recurrent, us"

In [5]:
k= ab.split(",")

In [6]:
z11=list(dict.fromkeys(k))

In [7]:
wordFre = dict.fromkeys(z11,0 )

In [8]:
wordFre

{'abrasion': 0,
 ' corneal abrasion': 0,
 ' corneal erosion': 0,
 ' ct': 0,
 ' left': 0,
 ' recurrent': 0,
 ' us': 0}